## A Very Simple ``Dash`` App

In [1]:
from dash import Dash, html, Output, Input

app = Dash()

app.layout = html.Div([
    html.H1("A Simple App", id="title"),
    html.P("Structure things like you would in HTML"),
    html.Button("Click me to do something", id="button")
])

@app.callback(
    Output("title", "children"), # change this value
    Input("button", "n_clicks"), # when this changes
    prevent_initial_call = True
)
def update_title_on_buttonclick(n_clicks):
    print(n_clicks) # sure why not
    return "A Simple App **That Does Things!**"

app.run()

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:8050
Press CTRL+C to quit
127.0.0.1 - - [22/Apr/2023 21:17:53] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [22/Apr/2023 21:17:53] "GET /_dash-component-suites/dash/deps/prop-types@15.v2_9_2m1680375717.8.1.min.js HTTP/1.1" 200 -
127.0.0.1 - - [22/Apr/2023 21:17:53] "GET /_dash-component-suites/dash/deps/polyfill@7.v2_9_2m1680375717.12.1.min.js HTTP/1.1" 200 -
127.0.0.1 - - [22/Apr/2023 21:17:53] "GET /_dash-component-suites/dash/deps/react-dom@16.v2_9_2m1680375717.14.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [22/Apr/2023 21:17:53] "GET /_dash-component-suites/dash/dash-renderer/build/dash_renderer.v2_9_2m1680375717.min.js HTTP/1.1" 200 -
127.0.0.1 - - [22/Apr/2023 21:17:53] "GET /_dash-component-suites/dash/dcc/dash_core_components.v2_9_1m1680375717.js HTTP/1.1" 200 -
127.0.0.1 - - [22/Apr/2023 21:17:53] "GET /_dash-component-suites/dash/dcc/dash_core_components-shared.v2_9_1m1680375717.js HTTP/1.1" 200 -
127.0.0.1 - - [22/Apr/2023 21:17:53] "GET /_dash-component-su

1


127.0.0.1 - - [22/Apr/2023 21:17:57] "POST /_dash-update-component HTTP/1.1" 200 -


2


127.0.0.1 - - [22/Apr/2023 21:17:57] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [22/Apr/2023 21:17:57] "POST /_dash-update-component HTTP/1.1" 200 -


3
4


127.0.0.1 - - [22/Apr/2023 21:17:58] "POST /_dash-update-component HTTP/1.1" 200 -


5


## A Bit More Complicated App

Let's use bootstrap to spruce this up a bit

In [3]:
import dash_bootstrap_components as dbc

app = Dash(external_stylesheets=[dbc.themes.SKETCHY])

app.layout = html.Div([
    dbc.NavbarSimple(brand="Communities in Network Graphs"),
    dbc.Container([
        dbc.Row([
            dbc.Col(
                "Hello", width=3
            ),
            dbc.Col("Everyone", width=9)
        ])
    ])
])

app.run()


Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:8050
Press CTRL+C to quit
127.0.0.1 - - [22/Apr/2023 21:22:24] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [22/Apr/2023 21:22:25] "GET /_dash-component-suites/dash_bootstrap_components/_components/dash_bootstrap_components.v1_4_1m1682212359.min.js HTTP/1.1" 200 -
127.0.0.1 - - [22/Apr/2023 21:22:26] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [22/Apr/2023 21:22:26] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [22/Apr/2023 21:22:26] "GET /_favicon.ico?v=2.9.2 HTTP/1.1" 200 -


## Cytoscape

Now we can use the `dash-cytoscape` package to display our graph. Let's start with wireframe layout and then add in the functionality we need:

In [4]:
from dash import dcc

app = Dash(external_stylesheets=[dbc.themes.SKETCHY])

app.layout = html.Div(
    [
        dbc.NavbarSimple(brand="Communities in Network Graphs"),
        dbc.Container(
            [dbc.Row([dbc.Col([
    dcc.Dropdown(options=["Manager", "Languages", "Team"], id="attributes", multi=True),
    dcc.Dropdown(options=["Team", "Job Title"], id="color_by"),
    ])])]
        ),
    ]
)

app.run()


Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:8050
Press CTRL+C to quit
127.0.0.1 - - [22/Apr/2023 21:27:00] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [22/Apr/2023 21:27:00] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [22/Apr/2023 21:27:00] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [22/Apr/2023 21:27:00] "GET /_favicon.ico?v=2.9.2 HTTP/1.1" 200 -
